In [1]:
#define hostname
HOSTNAME = '192.168.1.100'

In [2]:
import os
HOSTNAME = os.environ['REDPITAYA_HOSTNAME']

In [3]:
import pyrpl
p = pyrpl.Pyrpl(config="trig_test",  # do not use a config file 
                hostname=HOSTNAME)
rp = p.redpitaya  # shortcut for the the redpitaya handler

INFO:pyrpl:All your PyRPL settings will be saved to the config file
    /home/leo/pyrpl_user_dir/config/trig_test.yml
If you would like to restart PyRPL with these settings, type "pyrpl.exe trig_test" in a windows terminal or 
    from pyrpl import Pyrpl
    p = Pyrpl('trig_test')
in a python terminal.
INFO:pyrpl.redpitaya:Successfully connected to Redpitaya with hostname 169.254.137.54.


In [6]:
import numpy as np

def make_custom_waveform():
    waveform = np.zeros(2**14)
    for i in range(16):
        for j in range(100):
            waveform[i*1000+j] = i*0.01+j*0.008
        jmax = j
        vmax = waveform[i*1000+j]
        for j in range(100):
            waveform[i*1000+j+jmax] = vmax-j*0.007
    return waveform

# setup both asgs
for asg in [rp.asg0, 
            #rp.asg1
           ]:
    asg.setup(waveform='ramp', 
              amplitude=1.0, 
              offset=0, 
              frequency=1e5, 
              output_direct='off',
              trigger_source='immediately')
    asg.data = make_custom_waveform()
    
trig = rp.trig
trig.setup(input='asg0',
          output_direct='off',
          output_signal='max',
          threshold=0.1,
          hysteresis=0.005,
          auto_rearm=True,
          auto_rearm_delay=0.01/asg.frequency,
         )

# manually check on the scope
rp.scope.setup(input1='asg0',
               input2='trig',
               duration=5.0/asg.frequency,
               average=False,
               trigger_source='asg0',
               rolling_mode=False,
               running_state='running_continuous'
               )

In [24]:
trig._auto_rearm_delay=20000000

In [25]:
trig.armed=True

In [7]:
2**32/125e6

34.359738368

In [19]:
p.lockbox.inputs.port1.get_analog_offset?

ERROR:pyrpl.pyrpl_client:Error occured in reading attempt 0. Reconnecting at addr 0x40100000 to _writes value [1] by client 1
ERROR:quamash._QEventLoop:Exception in callback RunFuture._new_curve_arrived(<CurveFuture ...0.82385254]])>)
handle: <Handle RunFuture._new_curve_arrived(<CurveFuture ...0.82385254]])>)>
Traceback (most recent call last):
  File "/home/leo/github/pyrpl/pyrpl/pyrpl_client.py", line 186, in try_n_times
    value = function(addr, value)
  File "/home/leo/github/pyrpl/pyrpl/pyrpl_client.py", line 169, in _writes
    if self.socket.recv(8) == header:  # check for in-sync transmission
socket.timeout: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/leo/anaconda3/envs/pyrpl/lib/python3.6/asyncio/events.py", line 145, in _run
    self._callback(*self._args)
  File "/home/leo/github/pyrpl/pyrpl/acquisition_module.py", line 183, in _new_curve_arrived
    self.start()
  File "/home/leo/github/p